## 用户风险数据

'user'\
'score'\
'scenario'\
'events'\
'date'

In [1]:
events = {}
events['敏感数据泄露'] = '测试页面泄露在外网，管理后台地址泄漏，QQ群敏感文件泄露，百度网盘敏感信息泄露，github源码泄露，运维监控系统弱口令泄露，Http认证信息泄露'.split('，')
events['离职倾向用户风险'] = '长期停留在招聘网站，打开外部网页频率增加，文件上传百度网盘频率增加，某就业网站访问频率增加，网页查询消极信息频率增高'.split('，')
events['私人文件盗取'] = 'github源码频繁下载'.split('，')
events['管理员账号泄露'] = '账号异地登录，账号被多次输错后登录，账号不在工作时间内登录'.split('，')
events['被裁员用户风险'] = '被多次线上约谈'.split('，')

In [2]:
events

{'敏感数据泄露': ['测试页面泄露在外网',
  '管理后台地址泄漏',
  'QQ群敏感文件泄露',
  '百度网盘敏感信息泄露',
  'github源码泄露',
  '运维监控系统弱口令泄露',
  'Http认证信息泄露'],
 '离职倾向用户风险': ['长期停留在招聘网站',
  '打开外部网页频率增加',
  '文件上传百度网盘频率增加',
  '某就业网站访问频率增加',
  '网页查询消极信息频率增高'],
 '私人文件盗取': ['github源码频繁下载'],
 '管理员账号泄露': ['账号异地登录', '账号被多次输错后登录', '账号不在工作时间内登录'],
 '被裁员用户风险': ['被多次线上约谈']}

### event案例

In [3]:
import pandas as pd
import random
import time, json
from datetime import datetime
data = pd.read_excel('./用户.xlsx')

In [4]:
def strTimeProp(start, end, prop, frmt):
    stime = time.mktime(time.strptime(start, frmt))
    etime = time.mktime(time.strptime(end, frmt))
    ptime = stime + prop * (etime - stime)
    return int(ptime)


def randomDate(start, end, frmt='%Y-%m-%d %H:%M:%S'):
    return time.strftime(frmt, time.localtime(strTimeProp(start, end, random.random(), frmt)))

def randomDateList(start, end, n, frmt='%Y-%m-%d %H:%M:%S'):
    return [randomDate(start, end, frmt) for _ in range(n)]

start = '2021-09-01 00:00:00'
end = '2021-09-03 00:00:00'
lenth = 2
print(randomDateList(start, end, lenth))
randomdata = randomDateList(start, end, lenth)
datedict = {}
for i in randomdata:
    datedict[i] = random.sample(events['敏感数据泄露'], 1)[0]
datedict

['2021-09-02 18:21:31', '2021-09-02 18:24:02']


{'2021-09-01 14:37:23': '测试页面泄露在外网', '2021-09-01 06:53:21': 'QQ群敏感文件泄露'}

In [5]:
# 转json
json.dumps(datedict, ensure_ascii = False)

'{"2021-09-01 14:37:23": "测试页面泄露在外网", "2021-09-01 06:53:21": "QQ群敏感文件泄露"}'

### 单个用户案例

In [4]:
data.loc[0,:]

user    JLM0364
name      安全工程师
Name: 0, dtype: object

In [70]:
JLM0364 = pd.DataFrame()

In [71]:
JLM0364['datetime'] = pd.date_range(start = '2021-07-01 00:00:00', end = '2021-09-14 00:00:00', freq = '1D')
JLM0364['mingan_score'] = JLM0364.apply(lambda x: random.randint(10,74), axis = 1)
JLM0364['lizhi_score'] = JLM0364.apply(lambda x: random.randint(10,74), axis = 1)
JLM0364['siren_score'] = JLM0364.apply(lambda x: random.randint(10,74), axis = 1)
JLM0364['guanli_score'] = JLM0364.apply(lambda x: random.randint(10,74), axis = 1)
JLM0364['beicai_score'] = JLM0364.apply(lambda x: random.randint(10,74), axis = 1)
start_date = ['2021-09-01', '2021-09-04', '2021-09-06']
end_date = ['2021-09-03', '2021-09-05', '2021-09-08']
for i in range(len(start_date)):
    JLM0364['mingan_score'] = JLM0364.apply(lambda x: random.randint(75,90) if (str(x['datetime']) > start_date[i])& (str(x['datetime']) < end_date[i]) else x['mingan_score'], axis = 1)

In [72]:
JLM0364[JLM0364['mingan_score'] >= 75]

,datetime,mingan_score,lizhi_score,siren_score,guanli_score,beicai_score
62,2021-09-01,76,45,51,30,16
63,2021-09-02,88,44,50,47,69
65,2021-09-04,80,61,73,44,63
67,2021-09-06,75,16,39,38,51
68,2021-09-07,84,71,33,21,44


In [46]:
JLM0364_dict = JLM0364.to_json('JLM0364.json',orient = 'records')

In [63]:
with open('JLM0364.json','r',encoding='utf8')as fp:
    json_data = json.load(fp)

In [65]:
for line in json_data:
    line['timestamp'] = str(pd.to_datetime(datetime.datetime.fromtimestamp(int((line['datetime']-8*60*60*1000)/1000))))
    if line['mingan_score'] >= 75:
        start = line['timestamp']
        end = str(pd.to_datetime(datetime.datetime.fromtimestamp(int((line['datetime']-8*60*60*1000)/1000))+datetime.timedelta(days=1)))
        lenth = 2
        randomdata = randomDateList(start, end, lenth)
        datedict = {}
        for i in randomdata:
            datedict[i] = random.sample(events['敏感数据泄露'], 1)[0]
        line['mingan_event'] = json.dumps(datedict, ensure_ascii = False)
    else:
        line['mingan_event'] = ''

In [66]:
from elasticsearch import Elasticsearch
from elasticsearch import helpers

In [67]:
es = Elasticsearch('192.168.50.98:19201')

In [68]:
#批量更新也可以采用如下的方式进行json拼装，最后写入
ACTIONS = []
for line in json_data:
    action = {
        "_index": 'dc_emp_data',
        "_type": '_doc',
        "_source": {
            "user": 'JLM0364',
            "score": line['mingan_score'],
            "scenario": '敏感数据泄露',
            "events": line['mingan_event'],
            "datetime": line['timestamp']}
    }
    ACTIONS.append(action)
success, _ = helpers.bulk(es, ACTIONS, index='dc_emp_data', raise_on_error=True)

/home/weijunfei/anaconda3/lib/python3.7/site-packages/elasticsearch/connection/base.py:190: ElasticsearchDeprecationWarning: [types removal] Specifying types in bulk requests is deprecated.
  warnings.warn(message, category=ElasticsearchDeprecationWarning)
